# Projet Climatologie


## Liste des requêtes


- Les températures moyennes françaises de 1995 à 2019.

- L'évolution moyenne de la témparature chaque mois de l'année (sur toutes les années donc) de chaque région du globe (à l'aide de subplots, soit plusieurs diagrammes sur un seul affichage).

- Le classement graphique des pays les plus chauds au monde.

- Le classement graphique des pays les plus froids au monde.

- L'évolution des températures entre le pays le plus chaud et le pays le plus froid de 1995 à 2019 sur un même graphique.

- L'évolution de la température moyenne de la ville de Paris.

- La température moyenne de chaque pays sur une carte sur la période 2010-2019.



## Structure du jeu de données source

Region / Country / State / City / Month / Day / Year / AvgTemperature

## Import des librairies

In [2]:
# IPython extension reloading modules before user enters code.
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src.credentials import Credentials as cr
import src.split_datas as sd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Les températures moyennes françaises de 1995 à 2019

In [4]:
sd.split_datas(cr.TEMPERATURES, 'Country')

c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Algeria.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Burundi.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Benin.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Central African Republic.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Congo.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Egypt.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Ethiopia.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Gabon.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Gambia.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Guinea.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Guinea-Bissau.csv
c:\Users\utilisateur\Desktop\Climatologie\data\CURATED\Country\Ivory Coast.csv
c:\Users\utilisate